In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import os
from datetime import datetime
import datetime as dt

%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_seq_items', None)

# 한글 세팅
from matplotlib import font_manager, rc
font_path = "C:\Windows\Fonts/H2GTRM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [2]:
# 현재경로 확인
dir_path = os.getcwd()

# data 파일 경로 설정
file_name = 'DomesticCyberCrime_data.csv'
data_path = os.path.join(dir_path, file_name)
print(data_path)

C:\Users\swkim\OneDrive\바탕 화면\1. 개인\0. 공부\PYTHON\DomesticCyberCrime_data.csv


In [3]:
df = pd.read_csv(data_path)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\swkim\\OneDrive\\바탕 화면\\1. 개인\\0. 공부\\PYTHON\\DomesticCyberCrime_data.csv'

In [ ]:
df.info()

In [ ]:
cols = ['Year', 'Attack_Type', 'Attack_Skill', 'Count', 'Arrest_Count']
df.columns = cols
df

# EDA

In [ ]:
## 연도별 범죄유형별 사건수
df_year = df.set_index('Year')
df_year

In [ ]:
df_group = df.groupby(df['Year'])
df_group.sum()

In [ ]:
df_attack = df.groupby(df['Attack_Type'])
df_attack.sum()

## 정보통신망 침해 범죄 분석

In [ ]:
df1 = df[df['Attack_Type']=='정보통신망 침해']
df1['Attack_Skill'].value_counts

In [ ]:
df1

In [ ]:
df_year = pd.DataFrame(df['Year'].value_counts())
# df_year.sort_values(by=)
df_year.sort_index(inplace=True)

df_year.plot(figsize=(10,5))
plt.title("연도별 사이버범죄 건수")
plt.show()

# 01~20년도 사이버범죄 통계

In [ ]:
# 현재경로 확인
dir_path = os.getcwd()

# data 파일 경로 설정
file_name = 'stats_01,20.xlsx'
data_path = os.path.join(dir_path, file_name)
print(data_path)

In [ ]:
df_stats = pd.read_excel(data_path,engine="openpyxl")
df_stats.head()

In [ ]:
df_stats.info()

In [ ]:
cols = ['Year', "Count", 'Arrest_Count', 'Arrest_Rate']
df_stats.columns = cols
df_stats2 = df_stats.set_index('Year')
df_stats2

In [ ]:
df_stats2[['Count', 'Arrest_Count']].plot(figsize=(10,5))
plt.title("연도별 사이버범죄 건수")
plt.ylabel("사건 수")
plt.xticks(df_stats2.index)
plt.show()

## ARIMA를 통한 범죄 건수 예측

In [ ]:
## 데이터프레임 준비
df_arima = df_stats[['Year', 'Count']]
df_arima = df_arima.astype({'Year':'str'})
df_arima

In [ ]:
df_arima.Year = pd.to_datetime(df_arima.Year, format='%Y')
df_arima.set_index("Year", inplace=True)
df_arima

In [ ]:
df_arima.dropna(inplace=True)

In [ ]:
df_arima.plot()
plt.title("연도별 사이버범죄 건수")

### 시계열 분해
1. EDA를 통해 추세 및 변동요인을 확인
2. 시계열 분해로 확인
- 참고자료: https://leedakyeong.tistory.com/entry/%EC%8B%9C%EA%B3%84%EC%97%B4-%EB%B6%84%ED%95%B4%EB%9E%80Time-Series-Decomposition-%EC%8B%9C%EA%B3%84%EC%97%B4-%EB%B6%84%EC%84%9D%EC%9D%B4%EB%9E%80-%EC%8B%9C%EA%B3%84%EC%97%B4-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%9E%80-%EC%B6%94%EC%84%B8Trend-%EC%88%9C%ED%99%98Cycle-%EA%B3%84%EC%A0%88%EC%84%B1Seasonal-%EB%B6%88%EA%B7%9C%EC%B9%99-%EC%9A%94%EC%86%8CRandom-Residual

#### 패턴 확인
1. 점진적으로 상승하는 추세

In [ ]:
## 시계열 분해법
# 계절성 주기: 연도별 데이터이므로 의미없음
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_arima, model='additive', freq=4)

fig = plt.figure()
fig = result.plot()
fig.set_size_inches(20,15)

### 정상성 검정

In [ ]:
## 정상성 검정2
# 귀무가설: 자료가 정상성을 만족하지 않는다. => 채택
from statsmodels.tsa.stattools import adfuller
result = adfuller(df_arima)
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
print("Critical Values:")
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
# 1차 차분
ts_diff = df_arima - df_arima.shift()
ts_diff.plot()
plt.title("1차 차분 데이터")

In [ ]:
# 1차 차분 정상성 검정: 모수 중 d=1
# 귀무가설: 자료가 정상성을 만족하지 않는다. => 기각
result = adfuller(ts_diff[1:])
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
print("Critical Values:")
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
## 정상성 검정: True (만족)
from pmdarima.arima import ADFTest

adf_test = ADFTest(alpha=0.05)
adf_test.should_diff(ts_diff[1:])

###  ARIMA 분석
- 참고자료: https://otexts.com/fppkr/moving-averages.html
1. 모델 해석 (https://byeongkijeong.github.io/ARIMA-with-Python/)
- constant p값 확인=> 유효하다면 trend='c' 파라미터 지정
- MA(1)의 계수는 유효

In [ ]:

# 모수 결정
order = (2,1,1)

# 모델 생성
model = ARIMA(df_arima, order=order)

# 모델 학습
fit = model.fit(trend='c', disp=0)
print(fit.summary())

In [ ]:
pred = fit.predict(typ='levels')
pred.plot()

In [ ]:
fit.plot_predict()

In [ ]:
# 예측
fore = fit.forecast(steps=1)
print(fore)

In [ ]:
# 모수 결정
order = (1,1,1)

# 모델 생성
model = ARIMA(df_arima, order=order)

# 모델 학습
fit = model.fit(trend = 'c', full_output = True, disp = True)
print(fit.summary())

In [ ]:
pred = fit.predict(typ='levels')
pred

In [ ]:
plt.plot(pred)
# plt.plot(df_year)

In [ ]:
df_arima.plot()

## Auto ARIMA
- 참고자료: https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd
- document: https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html

1. 패턴 이해
2. 정상성 검정
3. 모델 생성
4. 모델 학습 및 예측

In [ ]:
## 정상성 검정: True (만족)
from pmdarima.arima import ADFTest

adf_test = ADFTest(alpha=0.05)
adf_test.should_diff(df_arima)

In [ ]:
## 정상성 검정2
# 귀무가설: 자료가 정상성을 만족하지 않는다. => 채택
from statsmodels.tsa.stattools import adfuller
result = adfuller(df_arima)
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
print("Critical Values:")
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
## AUTO ARIMA 분석
# 계절성이 없어 예측 잘 안되는 듯
from pmdarima.arima import auto_arima

arima_model = auto_arima(df_arima, start_p=2, start_q=2,
                           max_p=5, max_q=5, m=1,
                            seasonal=False,
                           d=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

In [ ]:
# 모델 요약
arima_model.summary()

In [ ]:
# 모델 예측
pred2 = pd.DataFrame(arima_model.predict(n_periods=20), index=df_arima.index)
pred2.columns = ['predicted_count']
pred2

In [ ]:
pred2['predicted_count']

In [ ]:
df_arima['Count']

In [ ]:
# 모델 평가
from sklearn.metrics import r2_score

r2_score(df_arima['Count'], pred2['predicted_count'])

### ARIMA 모수 결정
- ACF: Lag에 따른 관측치 사이 관련성 측정
- PACF: 다른 모든시점 영향력 배제하고, 두 관측치의 관련성 측정
> ACF, PACF가 급격히 감소하는 시차로 모수 결정

In [ ]:
## ACF, PACF plot
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(df_arima)
plot_pacf(df_arima['Count'])
plt.show()

In [ ]:
## 모수 (2,1,2) 모델 적용
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA

order = (2,1,2)
model = ARIMA(df_arima['Count'].values, order=order)
fit = model.fit(trend = 'c', full_output = True, disp = True)
print(fit.summary())

In [ ]:
order = (2,1,2)
model = ARIMA(df_arima, order)
fit = model.fit()
fit.summary()

In [ ]:
df_arima['Count'].values

## ARIMA 2차 시도

In [ ]:
# 데이터 프레임: 날짜형으로 변환하지 않음
df3 = df_stats[['Year', 'Count']]
df3.info()

In [ ]:
## 정상성 검정
# 귀무가설: 자료가 정상성을 만족하지 않는다. => 채택
from statsmodels.tsa.stattools import adfuller

result = adfuller(df3.Count)
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
print("Critical Values:")
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


In [ ]:
# 1차 차분
ts_diff = df3 - df3.shift()
ts_diff.plot()
plt.title("1차 차분 데이터")

In [ ]:
# 1차 차분 정상성 검정
# 귀무가설: 자료가 정상성을 만족하지 않는다. => 기각
result = adfuller(ts_diff[1:])
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
print("Critical Values:")
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


In [ ]:
# ARIMA 모델 적용
model = ARIMA(df3.Count, order=(2,1,2))
model_fit = model.fit()
model_fit.summary()

In [ ]:
df3

In [ ]:
# csv로 내보내기
dir_path
file_name = "data1007.csv"
data_path = os.path.join(dir_path, file_name)
df3.to_csv(data_path, sep=',', na_rep='NaN')